In [2]:
import os
import datetime
import warnings
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt


from scipy.stats import kurtosis


warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

In [3]:
class Bindfiles:
    
    @classmethod
    def read_files(cls, path, header=True):
        dirs = os.listdir(path)
        dfs = []
        for file in dirs:
            filename = path + '/' + file
            df = pd.read_csv(filename)
            dfs.append(df)
        df = pd.concat(dfs, ignore_index = True)
        return df 

    @classmethod
    def save_to_csv(cls, df, savename):
        df.to_csv(savename)
        return None

### 训练集

In [186]:
path1 = "D:/Jupyterfiles/202206CMP练习项目/CMP_data/DATA SET/CMP-data/training"
path2 = "D:/Jupyterfiles/202206CMP练习项目/CMP_data/VALIDATION DATA SET/validation"
df_training = Bindfiles.read_files(path1)
df_validation = Bindfiles.read_files(path2) 
df = pd.concat([df_training, df_validation], ignore_index = True)

In [187]:
f1 = open("D:/Jupyterfiles/202206CMP练习项目/CMP_data/DATA SET/CMP-training-removalrate.csv")
training_label = pd.read_csv(f1)

f2 = open("D:/Jupyterfiles/202206CMP练习项目/CMP_data/TestValidationAnswers/PHM16TestValidationAnswers/orig_CMP-validation-removalrate.csv")
validation_label = pd.read_csv(f2)

train_label = pd.concat([training_label, validation_label], ignore_index = True)

In [188]:
df_for_train =  pd.merge(df, train_label, on = ["WAFER_ID", "STAGE"], how = "outer")

In [189]:
df_for_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 816892 entries, 0 to 816891
Data columns (total 26 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   MACHINE_ID                    816892 non-null  object 
 1   MACHINE_DATA                  816892 non-null  object 
 2   TIMESTAMP                     816892 non-null  float64
 3   WAFER_ID                      816892 non-null  object 
 4   STAGE                         816892 non-null  object 
 5   CHAMBER                       816892 non-null  float64
 6   USAGE_OF_BACKING_FILM         816892 non-null  float64
 7   USAGE_OF_DRESSER              816892 non-null  float64
 8   USAGE_OF_POLISHING_TABLE      816892 non-null  float64
 9   USAGE_OF_DRESSER_TABLE        816892 non-null  float64
 10  PRESSURIZED_CHAMBER_PRESSURE  816892 non-null  float64
 11  MAIN_OUTER_AIR_BAG_PRESSURE   816892 non-null  float64
 12  CENTER_AIR_BAG_PRESSURE       816892 non-nul

### 测试集

In [207]:
path = "D:/Jupyterfiles/202206CMP练习项目/CMP_data/DATA SET/CMP-data/test"
df_test = Bindfiles.read_files(path)

f = open("D:/Jupyterfiles/202206CMP练习项目/CMP_data/TestValidationAnswers/PHM16TestValidationAnswers/orig_CMP-test-removalrate.csv")
test_label = pd.read_csv(f)

df_for_test = pd.merge(df_test, test_label, on = ["WAFER_ID", "STAGE"], how = "outer")

In [208]:
df_for_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156262 entries, 0 to 156261
Data columns (total 26 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   MACHINE_ID                    156262 non-null  object 
 1   MACHINE_DATA                  156262 non-null  object 
 2   TIMESTAMP                     156262 non-null  float64
 3   WAFER_ID                      156262 non-null  object 
 4   STAGE                         156262 non-null  object 
 5   CHAMBER                       156262 non-null  float64
 6   USAGE_OF_BACKING_FILM         156262 non-null  float64
 7   USAGE_OF_DRESSER              156262 non-null  float64
 8   USAGE_OF_POLISHING_TABLE      156262 non-null  float64
 9   USAGE_OF_DRESSER_TABLE        156262 non-null  float64
 10  PRESSURIZED_CHAMBER_PRESSURE  156262 non-null  float64
 11  MAIN_OUTER_AIR_BAG_PRESSURE   156262 non-null  float64
 12  CENTER_AIR_BAG_PRESSURE       156262 non-nul

### EDA

In [193]:
print(df_for_train['MACHINE_ID'].value_counts())
print(df_for_train['MACHINE_DATA'].value_counts())
print(df_for_train['STAGE'].value_counts())
print(df_for_train['CHAMBER'].value_counts())
print(df_for_train['DRESSING_WATER_STATUS'].value_counts())

2    816892
Name: MACHINE_ID, dtype: int64
4    376613
5    171108
6    147750
1     64611
3     31925
2     24885
Name: MACHINE_DATA, dtype: int64
A    459843
B    357049
Name: STAGE, dtype: int64
4.0    376613
5.0    171108
6.0    147750
1.0     64611
3.0     31925
2.0     24885
Name: CHAMBER, dtype: int64
0.0    471401
1.0    345491
Name: DRESSING_WATER_STATUS, dtype: int64


In [194]:
df_for_train.isnull().any()

MACHINE_ID               False
MACHINE_DATA             False
TIMESTAMP                False
WAFER_ID                 False
STAGE                    False
                         ...  
STAGE_ROTATION           False
HEAD_ROTATION            False
DRESSING_WATER_STATUS    False
EDGE_AIR_BAG_PRESSURE    False
AVG_REMOVAL_RATE         False
Length: 26, dtype: bool

In [195]:
def time_convert(x):
    d = datetime.datetime.fromtimestamp(x)
    str_d = d.strftime("%Y-%m-%d %H:%M:%S")
    return str_d

In [196]:
df_for_train['TIMESTAMP'] = list(map(time_convert, df_for_train['TIMESTAMP']))

In [197]:
df_for_train['MACHINE_ID'] = df_for_train['MACHINE_ID'].apply(int)
df_for_train['MACHINE_DATA'] = df_for_train['MACHINE_DATA'].apply(int)
df_for_train['WAFER_ID'] = df_for_train['WAFER_ID'].apply(int)
df_for_train['STAGE'] = df_for_train['STAGE'].apply(str)
df_for_train['CHAMBER'] = df_for_train['CHAMBER'].apply(int)
df_for_train['DRESSING_WATER_STATUS'] = df_for_train['DRESSING_WATER_STATUS'].apply(int)
df_for_train['TIMESTAMP'] = df_for_train['TIMESTAMP'].astype('datetime64[ns]')

In [198]:
df_for_train['AVG_REMOVAL_RATE'].value_counts().sort_index()

53.42655      353
53.47650      371
53.64735      346
53.69430      351
53.77380      355
             ... 
162.64170     277
4129.49400    266
4182.41655    267
4202.11245    536
4326.15405    253
Name: AVG_REMOVAL_RATE, Length: 2396, dtype: int64

In [199]:
df_for_train.shape

(816892, 26)

In [200]:
df_for_train1 = df_for_train.drop(df_for_train[df_for_train['AVG_REMOVAL_RATE'].isin([4129.49400, 4182.41655, 4202.11245, 4326.15405])].index)

df_for_train1 = df_for_train1.drop(['MACHINE_ID', 'MACHINE_DATA'], axis=1)

In [201]:
df_for_train1.shape

(815570, 24)

### 特征工程

#### （1）训练集处理

In [202]:
def feature_engineer(df, groupby_columns:list, label_column:list, time_column:list):
    
    # 时间特征
    time_feature = df.groupby(by=groupby_columns)[time_column].agg(['min', 'max'])
    time_feature.columns = time_feature.columns.map('_'.join)
    time_feature = time_feature.reset_index(drop=False)
    time_feature['duration'] = [(time_feature[time_column[0]+'_max'] - time_feature[time_column[0]+'_min'])[i].total_seconds() for i in range(len(time_feature))]
    time_feature = time_feature.drop(['TIMESTAMP_min', 'TIMESTAMP_max'], axis=1)

    # 数值统计特征
    feature_columns = list(set(df.columns) - set(groupby_columns) - set(label_column) - set(time_column))
    stats_features = df.groupby(by=groupby_columns)[feature_columns].agg(['mean', 'std', 'min', 'max', 'median', 'mad',
                                                                ('q25', lambda x:np.quantile(x, 0.25)),
                                                                ('q75', lambda x:np.quantile(x, 0.75))])
    stats_features.columns = stats_features.columns.map('_'.join)
    stats_features = stats_features.reset_index(drop=False)
    
    # 合并两个特征
    df_features = pd.merge(time_feature, stats_features, on = groupby_columns, how = "outer")
    df_features = df_features.reset_index(drop=True)
    return df_features

In [84]:
time1 = datetime.datetime.now()
groupby_columns = ['WAFER_ID', 'STAGE', 'CHAMBER', 'DRESSING_WATER_STATUS']
label_column = ['AVG_REMOVAL_RATE']
time_column = ['TIMESTAMP']

df_train_features = feature_engineer(df_for_train1, groupby_columns, label_column, time_column)
time2 = datetime.datetime.now()
print(time2-time1)

0:01:26.532075


In [203]:
print(df_train_features.shape)
df_train_features

(10810, 149)


,WAFER_ID,STAGE,CHAMBER,DRESSING_WATER_STATUS,duration,MAIN_OUTER_AIR_BAG_PRESSURE_mean,MAIN_OUTER_AIR_BAG_PRESSURE_std,MAIN_OUTER_AIR_BAG_PRESSURE_min,MAIN_OUTER_AIR_BAG_PRESSURE_max,MAIN_OUTER_AIR_BAG_PRESSURE_median,...,EDGE_AIR_BAG_PRESSURE_q25,EDGE_AIR_BAG_PRESSURE_q75,SLURRY_FLOW_LINE_A_mean,SLURRY_FLOW_LINE_A_std,SLURRY_FLOW_LINE_A_min,SLURRY_FLOW_LINE_A_max,SLURRY_FLOW_LINE_A_median,SLURRY_FLOW_LINE_A_mad,SLURRY_FLOW_LINE_A_q25,SLURRY_FLOW_LINE_A_q75
0,-4230160606,B,4,0,161.0,237.644444,60.818745,0.0,258.0,258.0,...,43.939394,44.242424,4.787380,6.634060,2.222222,25.833333,2.222222,4.306826e+00,2.222222,2.222222
1,-4230160606,B,4,1,204.0,2.857143,12.932416,0.0,60.0,0.0,...,0.000000,0.000000,7.308201,7.378823,2.222222,22.916667,2.222222,6.133157e+00,2.222222,10.312500
2,-4230160606,B,5,0,84.0,229.482353,79.858227,0.0,258.0,258.0,...,43.939394,44.242424,3.089869,3.167510,2.222222,21.944444,2.222222,1.524606e+00,2.222222,2.222222
3,-4230160606,B,6,0,70.0,246.616667,21.804658,136.8,258.0,256.8,...,43.939394,45.530303,6.709105,8.690370,2.222222,25.833333,2.222222,6.968021e+00,2.222222,2.222222
4,-4230160606,B,6,1,0.0,216.000000,NaN,216.0,216.0,216.0,...,57.575758,57.575758,25.972222,NaN,25.972222,25.972222,25.972222,0.000000e+00,25.972222,25.972222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10805,4229773758,B,4,0,167.0,220.428571,85.315646,0.0,258.0,258.0,...,43.939394,44.242424,4.228671,6.274427,0.000000,26.388889,2.222222,3.633285e+00,2.222222,2.222222
10806,4229773758,B,4,1,200.0,9.418182,35.653484,0.0,190.8,0.0,...,0.000000,0.000000,4.827441,5.367704,2.222222,25.000000,2.222222,3.423375e+00,2.222222,4.444444
10807,4229773758,B,5,0,83.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,3.604651,4.282163,2.222222,23.333333,2.222222,2.298465e+00,2.222222,2.222222
10808,4229773758,B,5,1,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,23.263889,0.098209,23.194444,23.333333,23.263889,6.944444e-02,23.229167,23.298611


In [86]:
print(df_train_features['WAFER_ID'].value_counts())
print(df_train_features['STAGE'].value_counts())
print(df_train_features['CHAMBER'].value_counts())
print(df_train_features['DRESSING_WATER_STATUS'].value_counts())
print(df_train_features['duration'].value_counts())

 3621681198    12
 1300253518    12
 1462254328    11
 1286253712    11
 3627681150    11
               ..
-4033511544     3
 35494392       3
 1308253554     3
 2985014372     3
 3015014228     2
Name: WAFER_ID, Length: 1996, dtype: int64
A    6540
B    4270
Name: STAGE, dtype: int64
4    3323
6    2870
5    2300
1     812
3     773
2     732
Name: CHAMBER, dtype: int64
1    5616
0    5194
Name: DRESSING_WATER_STATUS, dtype: int64
84.0     715
83.0     527
71.0     402
70.0     398
85.0     355
        ... 
573.0      1
568.0      1
274.0      1
92.0       1
456.0      1
Name: duration, Length: 243, dtype: int64


In [92]:
missing = df_train_features.isna().sum()
missing = missing[missing>0]
missing.sort_values(inplace=True)
missing

MAIN_OUTER_AIR_BAG_PRESSURE_std     258
STAGE_ROTATION_std                  258
CENTER_AIR_BAG_PRESSURE_std         258
USAGE_OF_DRESSER_TABLE_std          258
USAGE_OF_MEMBRANE_std               258
RIPPLE_AIR_BAG_PRESSURE_std         258
HEAD_ROTATION_std                   258
RETAINER_RING_PRESSURE_std          258
USAGE_OF_POLISHING_TABLE_std        258
SLURRY_FLOW_LINE_B_std              258
WAFER_ROTATION_std                  258
USAGE_OF_DRESSER_std                258
USAGE_OF_PRESSURIZED_SHEET_std      258
SLURRY_FLOW_LINE_C_std              258
PRESSURIZED_CHAMBER_PRESSURE_std    258
USAGE_OF_BACKING_FILM_std           258
EDGE_AIR_BAG_PRESSURE_std           258
SLURRY_FLOW_LINE_A_std              258
dtype: int64

In [155]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

# 查看缺失值所在的数据
df_missing = df_train_features[df_train_features.isna().any(axis=1)]

# cols_m = df_missing.columns[df_missing.columns.str.contains('min|max|median', case=False)]
# df_missing[cols_m]
df_missing

,WAFER_ID,STAGE,CHAMBER,DRESSING_WATER_STATUS,duration,MAIN_OUTER_AIR_BAG_PRESSURE_mean,MAIN_OUTER_AIR_BAG_PRESSURE_std,MAIN_OUTER_AIR_BAG_PRESSURE_min,MAIN_OUTER_AIR_BAG_PRESSURE_max,MAIN_OUTER_AIR_BAG_PRESSURE_median,...,EDGE_AIR_BAG_PRESSURE_q25,EDGE_AIR_BAG_PRESSURE_q75,SLURRY_FLOW_LINE_A_mean,SLURRY_FLOW_LINE_A_std,SLURRY_FLOW_LINE_A_min,SLURRY_FLOW_LINE_A_max,SLURRY_FLOW_LINE_A_median,SLURRY_FLOW_LINE_A_mad,SLURRY_FLOW_LINE_A_q25,SLURRY_FLOW_LINE_A_q75
4,-4230160606,B,6,1,0.0,216.0,NaN,216.0,216.0,216.0,...,57.575758,57.575758,25.972222,NaN,25.972222,25.972222,25.972222,0.0,25.972222,25.972222
36,-4230160424,B,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.000000,0.000000,23.333333,NaN,23.333333,23.333333,23.333333,0.0,23.333333,23.333333
87,-4228160690,A,6,1,0.0,217.2,NaN,217.2,217.2,217.2,...,57.878788,57.878788,25.138889,NaN,25.138889,25.138889,25.138889,0.0,25.138889,25.138889
94,-4228160686,A,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.000000,0.000000,22.916667,NaN,22.916667,22.916667,22.916667,0.0,22.916667,22.916667
165,-4228160600,B,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.000000,0.000000,23.194444,NaN,23.194444,23.194444,23.194444,0.0,23.194444,23.194444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10718,4227773654,B,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.000000,0.000000,22.916667,NaN,22.916667,22.916667,22.916667,0.0,22.916667,22.916667
10766,4229773698,B,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.000000,0.000000,23.611111,NaN,23.611111,23.611111,23.611111,0.0,23.611111,23.611111
10789,4229773742,B,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.000000,0.000000,23.055556,NaN,23.055556,23.055556,23.055556,0.0,23.055556,23.055556
10798,4229773746,B,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.000000,0.000000,22.638889,NaN,22.638889,22.638889,22.638889,0.0,22.638889,22.638889


In [157]:
# 查看含有缺失值的WAFER_ID和这些WAFER_ID在df_train_features有多少条记录
uniq = df_missing.WAFER_ID.value_counts().reset_index()
uniq.columns = ['WAFER_ID', 'value_counts_missing']

num_count = []
for wafer in uniq['WAFER_ID'].to_list():
    dfi = df_train_features[df_train_features['WAFER_ID'] == wafer]
    num_count.append(len(dfi))

uniq['value_counts_features'] = num_count
uniq

,WAFER_ID,value_counts_missing,value_counts_features
0,4229773746,2,10
1,1290253744,2,11
2,-4216160640,2,11
3,2977014410,2,10
4,1324253768,2,11
...,...,...,...
244,1288253462,1,5
245,1288253490,1,9
246,1290253662,1,8
247,1290253724,1,5


In [158]:
uniq.query("value_counts_missing == value_counts_features")

,WAFER_ID,value_counts_missing,value_counts_features


In [160]:
# 按照行去掉缺失值
print(df_train_features.shape)
df_train_features_dropna = df_train_features.dropna(axis=0, how='any')
print(df_train_features_dropna.shape)

(10810, 149)
(10552, 149)


In [162]:
df_train_features_dropna

,WAFER_ID,STAGE,CHAMBER,DRESSING_WATER_STATUS,duration,MAIN_OUTER_AIR_BAG_PRESSURE_mean,MAIN_OUTER_AIR_BAG_PRESSURE_std,MAIN_OUTER_AIR_BAG_PRESSURE_min,MAIN_OUTER_AIR_BAG_PRESSURE_max,MAIN_OUTER_AIR_BAG_PRESSURE_median,...,EDGE_AIR_BAG_PRESSURE_q25,EDGE_AIR_BAG_PRESSURE_q75,SLURRY_FLOW_LINE_A_mean,SLURRY_FLOW_LINE_A_std,SLURRY_FLOW_LINE_A_min,SLURRY_FLOW_LINE_A_max,SLURRY_FLOW_LINE_A_median,SLURRY_FLOW_LINE_A_mad,SLURRY_FLOW_LINE_A_q25,SLURRY_FLOW_LINE_A_q75
0,-4230160606,B,4,0,161.0,237.644444,60.818745,0.0,258.0,258.0,...,43.939394,44.242424,4.787380,6.634060,2.222222,25.833333,2.222222,4.306826e+00,2.222222,2.222222
1,-4230160606,B,4,1,204.0,2.857143,12.932416,0.0,60.0,0.0,...,0.000000,0.000000,7.308201,7.378823,2.222222,22.916667,2.222222,6.133157e+00,2.222222,10.312500
2,-4230160606,B,5,0,84.0,229.482353,79.858227,0.0,258.0,258.0,...,43.939394,44.242424,3.089869,3.167510,2.222222,21.944444,2.222222,1.524606e+00,2.222222,2.222222
3,-4230160606,B,6,0,70.0,246.616667,21.804658,136.8,258.0,256.8,...,43.939394,45.530303,6.709105,8.690370,2.222222,25.833333,2.222222,6.968021e+00,2.222222,2.222222
5,-4230160598,A,4,0,144.0,242.151724,71.554720,0.0,270.0,268.8,...,48.484848,48.787879,5.171456,7.128578,0.000000,26.250000,2.222222,4.820637e+00,2.222222,2.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10805,4229773758,B,4,0,167.0,220.428571,85.315646,0.0,258.0,258.0,...,43.939394,44.242424,4.228671,6.274427,0.000000,26.388889,2.222222,3.633285e+00,2.222222,2.222222
10806,4229773758,B,4,1,200.0,9.418182,35.653484,0.0,190.8,0.0,...,0.000000,0.000000,4.827441,5.367704,2.222222,25.000000,2.222222,3.423375e+00,2.222222,4.444444
10807,4229773758,B,5,0,83.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,3.604651,4.282163,2.222222,23.333333,2.222222,2.298465e+00,2.222222,2.222222
10808,4229773758,B,5,1,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,23.263889,0.098209,23.194444,23.333333,23.263889,6.944444e-02,23.229167,23.298611


In [176]:
train_data =  pd.merge(df_train_features_dropna, train_label, on = ["WAFER_ID", "STAGE"], how = "left")
print(train_data.isna().any().any())

False


In [239]:
train_data

,WAFER_ID,STAGE,CHAMBER,DRESSING_WATER_STATUS,duration,MAIN_OUTER_AIR_BAG_PRESSURE_mean,MAIN_OUTER_AIR_BAG_PRESSURE_std,MAIN_OUTER_AIR_BAG_PRESSURE_min,MAIN_OUTER_AIR_BAG_PRESSURE_max,MAIN_OUTER_AIR_BAG_PRESSURE_median,...,EDGE_AIR_BAG_PRESSURE_q75,SLURRY_FLOW_LINE_A_mean,SLURRY_FLOW_LINE_A_std,SLURRY_FLOW_LINE_A_min,SLURRY_FLOW_LINE_A_max,SLURRY_FLOW_LINE_A_median,SLURRY_FLOW_LINE_A_mad,SLURRY_FLOW_LINE_A_q25,SLURRY_FLOW_LINE_A_q75,AVG_REMOVAL_RATE
0,-4230160606,B,4,0,161.0,237.644444,60.818745,0.0,258.0,258.0,...,44.242424,4.787380,6.634060,2.222222,25.833333,2.222222,4.306826e+00,2.222222,2.222222,56.29485
1,-4230160606,B,4,1,204.0,2.857143,12.932416,0.0,60.0,0.0,...,0.000000,7.308201,7.378823,2.222222,22.916667,2.222222,6.133157e+00,2.222222,10.312500,56.29485
2,-4230160606,B,5,0,84.0,229.482353,79.858227,0.0,258.0,258.0,...,44.242424,3.089869,3.167510,2.222222,21.944444,2.222222,1.524606e+00,2.222222,2.222222,56.29485
3,-4230160606,B,6,0,70.0,246.616667,21.804658,136.8,258.0,256.8,...,45.530303,6.709105,8.690370,2.222222,25.833333,2.222222,6.968021e+00,2.222222,2.222222,56.29485
4,-4230160598,A,4,0,144.0,242.151724,71.554720,0.0,270.0,268.8,...,48.787879,5.171456,7.128578,0.000000,26.250000,2.222222,4.820637e+00,2.222222,2.222222,68.88180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10547,4229773758,B,4,0,167.0,220.428571,85.315646,0.0,258.0,258.0,...,44.242424,4.228671,6.274427,0.000000,26.388889,2.222222,3.633285e+00,2.222222,2.222222,66.56505
10548,4229773758,B,4,1,200.0,9.418182,35.653484,0.0,190.8,0.0,...,0.000000,4.827441,5.367704,2.222222,25.000000,2.222222,3.423375e+00,2.222222,4.444444,66.56505
10549,4229773758,B,5,0,83.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,3.604651,4.282163,2.222222,23.333333,2.222222,2.298465e+00,2.222222,2.222222,66.56505
10550,4229773758,B,5,1,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,23.263889,0.098209,23.194444,23.333333,23.263889,6.944444e-02,23.229167,23.298611,66.56505


In [244]:
train_data = pd.get_dummies(train_data, columns=['STAGE'])
train_data

,WAFER_ID,CHAMBER,DRESSING_WATER_STATUS,duration,MAIN_OUTER_AIR_BAG_PRESSURE_mean,MAIN_OUTER_AIR_BAG_PRESSURE_std,MAIN_OUTER_AIR_BAG_PRESSURE_min,MAIN_OUTER_AIR_BAG_PRESSURE_max,MAIN_OUTER_AIR_BAG_PRESSURE_median,MAIN_OUTER_AIR_BAG_PRESSURE_mad,...,SLURRY_FLOW_LINE_A_std,SLURRY_FLOW_LINE_A_min,SLURRY_FLOW_LINE_A_max,SLURRY_FLOW_LINE_A_median,SLURRY_FLOW_LINE_A_mad,SLURRY_FLOW_LINE_A_q25,SLURRY_FLOW_LINE_A_q75,AVG_REMOVAL_RATE,STAGE_A,STAGE_B
0,-4230160606,4,0,161.0,237.644444,60.818745,0.0,258.0,258.0,33.274074,...,6.634060,2.222222,25.833333,2.222222,4.306826e+00,2.222222,2.222222,56.29485,0,1
1,-4230160606,4,1,204.0,2.857143,12.932416,0.0,60.0,0.0,5.442177,...,7.378823,2.222222,22.916667,2.222222,6.133157e+00,2.222222,10.312500,56.29485,0,1
2,-4230160606,5,0,84.0,229.482353,79.858227,0.0,258.0,258.0,49.704083,...,3.167510,2.222222,21.944444,2.222222,1.524606e+00,2.222222,2.222222,56.29485,0,1
3,-4230160606,6,0,70.0,246.616667,21.804658,136.8,258.0,256.8,16.691204,...,8.690370,2.222222,25.833333,2.222222,6.968021e+00,2.222222,2.222222,56.29485,0,1
4,-4230160598,4,0,144.0,242.151724,71.554720,0.0,270.0,268.8,44.080666,...,7.128578,0.000000,26.250000,2.222222,4.820637e+00,2.222222,2.222222,68.88180,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10547,4229773758,4,0,167.0,220.428571,85.315646,0.0,258.0,258.0,57.356122,...,6.274427,0.000000,26.388889,2.222222,3.633285e+00,2.222222,2.222222,66.56505,0,1
10548,4229773758,4,1,200.0,9.418182,35.653484,0.0,190.8,0.0,17.123967,...,5.367704,2.222222,25.000000,2.222222,3.423375e+00,2.222222,4.444444,66.56505,0,1
10549,4229773758,5,0,83.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,4.282163,2.222222,23.333333,2.222222,2.298465e+00,2.222222,2.222222,66.56505,0,1
10550,4229773758,5,1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.098209,23.194444,23.333333,23.263889,6.944444e-02,23.229167,23.298611,66.56505,0,1


In [304]:
train_data[train_data['WAFER_ID'] == -4230160606]

,WAFER_ID,CHAMBER,DRESSING_WATER_STATUS,duration,MAIN_OUTER_AIR_BAG_PRESSURE_mean,MAIN_OUTER_AIR_BAG_PRESSURE_std,MAIN_OUTER_AIR_BAG_PRESSURE_min,MAIN_OUTER_AIR_BAG_PRESSURE_max,MAIN_OUTER_AIR_BAG_PRESSURE_median,MAIN_OUTER_AIR_BAG_PRESSURE_mad,...,SLURRY_FLOW_LINE_A_std,SLURRY_FLOW_LINE_A_min,SLURRY_FLOW_LINE_A_max,SLURRY_FLOW_LINE_A_median,SLURRY_FLOW_LINE_A_mad,SLURRY_FLOW_LINE_A_q25,SLURRY_FLOW_LINE_A_q75,AVG_REMOVAL_RATE,STAGE_A,STAGE_B
0,-4230160606,4,0,161.0,237.644444,60.818745,0.0,258.0,258.0,33.274074,...,6.634060,2.222222,25.833333,2.222222,4.306826,2.222222,2.222222,56.29485,0,1
1,-4230160606,4,1,204.0,2.857143,12.932416,0.0,60.0,0.0,5.442177,...,7.378823,2.222222,22.916667,2.222222,6.133157,2.222222,10.312500,56.29485,0,1
2,-4230160606,5,0,84.0,229.482353,79.858227,0.0,258.0,258.0,49.704083,...,3.167510,2.222222,21.944444,2.222222,1.524606,2.222222,2.222222,56.29485,0,1
3,-4230160606,6,0,70.0,246.616667,21.804658,136.8,258.0,256.8,16.691204,...,8.690370,2.222222,25.833333,2.222222,6.968021,2.222222,2.222222,56.29485,0,1


#### （2）测试集处理

In [209]:
df_for_test['TIMESTAMP'] = list(map(time_convert, df_for_test['TIMESTAMP']))

df_for_test['WAFER_ID'] = df_for_test['WAFER_ID'].apply(int)
df_for_test['STAGE'] = df_for_test['STAGE'].apply(str)
df_for_test['CHAMBER'] = df_for_test['CHAMBER'].apply(int)
df_for_test['DRESSING_WATER_STATUS'] = df_for_test['DRESSING_WATER_STATUS'].apply(int)
df_for_test['TIMESTAMP'] = df_for_test['TIMESTAMP'].astype('datetime64[ns]')
df_for_test1 = df_for_test.drop(['MACHINE_ID', 'MACHINE_DATA'], axis=1)

In [210]:
df_for_test1

,TIMESTAMP,WAFER_ID,STAGE,CHAMBER,USAGE_OF_BACKING_FILM,USAGE_OF_DRESSER,USAGE_OF_POLISHING_TABLE,USAGE_OF_DRESSER_TABLE,PRESSURIZED_CHAMBER_PRESSURE,MAIN_OUTER_AIR_BAG_PRESSURE,...,USAGE_OF_PRESSURIZED_SHEET,SLURRY_FLOW_LINE_A,SLURRY_FLOW_LINE_B,SLURRY_FLOW_LINE_C,WAFER_ROTATION,STAGE_ROTATION,HEAD_ROTATION,DRESSING_WATER_STATUS,EDGE_AIR_BAG_PRESSURE,AVG_REMOVAL_RATE
0,1985-04-06 20:40:28,373446766,A,1,9356.666667,536.296296,341.481481,2667.75,0.000000,0.0,...,2807.0,0.000000,0.000000,0.0,0.0,0.0,156.8,1,0.0,153.2610
1,1985-04-06 20:40:29,373446766,A,1,9356.666667,536.296296,341.481481,2667.75,0.000000,0.0,...,2807.0,0.000000,0.000000,0.0,0.0,0.0,156.8,1,0.0,153.2610
2,1985-04-06 20:40:30,373446766,A,1,9356.666667,536.296296,341.481481,2667.75,0.000000,0.0,...,2807.0,0.000000,0.000000,0.0,0.0,0.0,156.8,1,0.0,153.2610
3,1985-04-06 20:40:31,373446766,A,1,9356.666667,536.296296,341.481481,2667.75,0.000000,0.0,...,2807.0,0.000000,0.000000,0.0,0.0,0.0,156.8,1,0.0,153.2610
4,1985-04-06 20:40:32,373446766,A,1,9356.666667,536.296296,341.481481,2667.75,0.000000,0.0,...,2807.0,0.000000,0.000000,0.0,0.0,0.0,160.0,1,0.0,153.2610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156257,1985-06-10 09:53:18,33494174,A,6,356.666667,651.481481,349.629630,4293.50,20.952381,0.0,...,107.0,2.222222,0.909091,0.0,0.0,0.0,160.0,1,0.0,70.7889
156258,1985-06-10 09:53:19,33494174,A,6,356.666667,651.481481,349.629630,4293.50,20.476190,0.0,...,107.0,2.222222,0.909091,0.0,0.0,0.0,160.0,1,0.0,70.7889
156259,1985-06-10 09:53:20,33494174,A,6,356.666667,651.481481,349.629630,4293.50,20.952381,0.0,...,107.0,2.222222,0.909091,0.0,0.0,0.0,160.0,1,0.0,70.7889
156260,1985-06-10 09:53:21,33494174,A,6,356.666667,651.481481,349.629630,4293.50,28.571429,0.0,...,107.0,2.222222,0.909091,0.0,0.0,0.0,156.8,1,0.0,70.7889


In [211]:
time1 = datetime.datetime.now()
groupby_columns = ['WAFER_ID', 'STAGE', 'CHAMBER', 'DRESSING_WATER_STATUS']
label_column = ['AVG_REMOVAL_RATE']
time_column = ['TIMESTAMP']

df_test_features = feature_engineer(df_for_test1, groupby_columns, label_column, time_column)
time2 = datetime.datetime.now()
print(time2-time1)

0:00:14.611570


In [212]:
df_test_features

,WAFER_ID,STAGE,CHAMBER,DRESSING_WATER_STATUS,duration,MAIN_OUTER_AIR_BAG_PRESSURE_mean,MAIN_OUTER_AIR_BAG_PRESSURE_std,MAIN_OUTER_AIR_BAG_PRESSURE_min,MAIN_OUTER_AIR_BAG_PRESSURE_max,MAIN_OUTER_AIR_BAG_PRESSURE_median,...,EDGE_AIR_BAG_PRESSURE_q25,EDGE_AIR_BAG_PRESSURE_q75,SLURRY_FLOW_LINE_A_mean,SLURRY_FLOW_LINE_A_std,SLURRY_FLOW_LINE_A_min,SLURRY_FLOW_LINE_A_max,SLURRY_FLOW_LINE_A_median,SLURRY_FLOW_LINE_A_mad,SLURRY_FLOW_LINE_A_q25,SLURRY_FLOW_LINE_A_q75
0,-4226160404,A,4,0,151.0,245.289474,66.305305,0.0,270.0,268.8,...,48.484848,48.484848,4.966192,6.930228,0.000000,26.250000,2.222222,4.524120,2.222222,2.222222
1,-4226160404,A,4,1,197.0,9.808696,37.368054,0.0,216.0,0.0,...,0.000000,0.000000,7.774758,8.079811,2.222222,26.388889,2.361111,6.861741,2.222222,13.402778
2,-4226160404,A,5,0,84.0,261.967059,41.582836,0.0,270.0,268.8,...,48.484848,48.484848,2.357843,0.512545,2.222222,5.277778,2.222222,0.248904,2.222222,2.222222
3,-4226160404,A,6,0,53.0,250.511111,30.915629,114.0,270.0,268.8,...,48.484848,57.575758,8.382202,9.758698,2.222222,26.250000,2.222222,8.592440,2.222222,17.743056
4,-4226160404,A,6,1,19.0,22.020000,52.495209,0.0,200.4,0.0,...,0.000000,0.000000,7.138889,6.841399,2.222222,25.972222,4.166667,5.050000,2.222222,8.194444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1901,4227773662,A,6,0,73.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1902,4229773718,B,4,0,178.0,222.021053,84.233773,0.0,258.0,258.0,...,43.939394,43.939394,4.517544,6.959906,2.222222,26.250000,2.222222,4.134264,2.222222,2.222222
1903,4229773718,B,4,1,210.0,22.290000,56.922871,0.0,216.0,0.0,...,0.000000,0.000000,6.947917,7.488812,2.222222,26.250000,3.472222,5.511979,2.222222,7.152778
1904,4229773718,B,5,0,85.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.208333,1.932004,0.000000,17.916667,0.000000,0.411822,0.000000,0.000000


In [213]:
missing = df_test_features.isna().sum()
missing = missing[missing>0]
missing.sort_values(inplace=True)
missing

MAIN_OUTER_AIR_BAG_PRESSURE_std     44
STAGE_ROTATION_std                  44
CENTER_AIR_BAG_PRESSURE_std         44
USAGE_OF_DRESSER_TABLE_std          44
USAGE_OF_MEMBRANE_std               44
RIPPLE_AIR_BAG_PRESSURE_std         44
HEAD_ROTATION_std                   44
RETAINER_RING_PRESSURE_std          44
USAGE_OF_POLISHING_TABLE_std        44
SLURRY_FLOW_LINE_B_std              44
WAFER_ROTATION_std                  44
USAGE_OF_DRESSER_std                44
USAGE_OF_PRESSURIZED_SHEET_std      44
SLURRY_FLOW_LINE_C_std              44
PRESSURIZED_CHAMBER_PRESSURE_std    44
USAGE_OF_BACKING_FILM_std           44
EDGE_AIR_BAG_PRESSURE_std           44
SLURRY_FLOW_LINE_A_std              44
dtype: int64

In [216]:
# 查看缺失值所在的数据
df_missing = df_test_features[df_test_features.isna().any(axis=1)]

# cols_m = df_missing.columns[df_missing.columns.str.contains('min|max|median', case=False)]
# df_missing[cols_m]
df_missing

,WAFER_ID,STAGE,CHAMBER,DRESSING_WATER_STATUS,duration,MAIN_OUTER_AIR_BAG_PRESSURE_mean,MAIN_OUTER_AIR_BAG_PRESSURE_std,MAIN_OUTER_AIR_BAG_PRESSURE_min,MAIN_OUTER_AIR_BAG_PRESSURE_max,MAIN_OUTER_AIR_BAG_PRESSURE_median,...,EDGE_AIR_BAG_PRESSURE_q25,EDGE_AIR_BAG_PRESSURE_q75,SLURRY_FLOW_LINE_A_mean,SLURRY_FLOW_LINE_A_std,SLURRY_FLOW_LINE_A_min,SLURRY_FLOW_LINE_A_max,SLURRY_FLOW_LINE_A_median,SLURRY_FLOW_LINE_A_mad,SLURRY_FLOW_LINE_A_q25,SLURRY_FLOW_LINE_A_q75
99,-4035511566,B,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,22.777778,NaN,22.777778,22.777778,22.777778,0.0,22.777778,22.777778
112,-4033511540,A,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,22.638889,NaN,22.638889,22.638889,22.638889,0.0,22.638889,22.638889
124,-4025511544,B,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,23.194444,NaN,23.194444,23.194444,23.194444,0.0,23.194444,23.194444
144,-4021511610,A,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,23.194444,NaN,23.194444,23.194444,23.194444,0.0,23.194444,23.194444
156,-4019511616,A,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,23.194444,NaN,23.194444,23.194444,23.194444,0.0,23.194444,23.194444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,4167773600,B,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,22.638889,NaN,22.638889,22.638889,22.638889,0.0,22.638889,22.638889
1797,4201773448,A,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,22.777778,NaN,22.777778,22.777778,22.777778,0.0,22.777778,22.777778
1858,4217773676,A,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,23.194444,NaN,23.194444,23.194444,23.194444,0.0,23.194444,23.194444
1877,4221773734,B,5,1,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,23.333333,NaN,23.333333,23.333333,23.333333,0.0,23.333333,23.333333


In [217]:
# 查看含有缺失值的WAFER_ID和这些WAFER_ID在df_test_features有多少条记录
uniq = df_missing.WAFER_ID.value_counts().reset_index()
uniq.columns = ['WAFER_ID', 'value_counts_missing']

num_count = []
for wafer in uniq['WAFER_ID'].to_list():
    dfi = df_test_features[df_test_features['WAFER_ID'] == wafer]
    num_count.append(len(dfi))

uniq['value_counts_features'] = num_count
uniq 

,WAFER_ID,value_counts_missing,value_counts_features
0,-4035511566,1,6
1,-4033511540,1,6
2,1475739272,1,5
3,1488254354,1,4
4,1490254136,1,6
...,...,...,...
39,1270253696,1,5
40,1306253820,1,5
41,1314253600,1,6
42,1316253842,1,6


In [219]:
uniq.query("value_counts_missing == value_counts_features")

,WAFER_ID,value_counts_missing,value_counts_features


In [220]:
# 按照行去掉缺失值
print(df_test_features.shape)
df_test_features_dropna = df_test_features.dropna(axis=0, how='any')
print(df_test_features_dropna.shape)

(1906, 149)
(1862, 149)


In [247]:
test_data =  pd.merge(df_test_features_dropna, test_label, on = ["WAFER_ID", "STAGE"], how = "left")
print(test_data.isna().any().any())

False


In [248]:
test_data = pd.get_dummies(test_data, columns=['STAGE'])
test_data

,WAFER_ID,CHAMBER,DRESSING_WATER_STATUS,duration,MAIN_OUTER_AIR_BAG_PRESSURE_mean,MAIN_OUTER_AIR_BAG_PRESSURE_std,MAIN_OUTER_AIR_BAG_PRESSURE_min,MAIN_OUTER_AIR_BAG_PRESSURE_max,MAIN_OUTER_AIR_BAG_PRESSURE_median,MAIN_OUTER_AIR_BAG_PRESSURE_mad,...,SLURRY_FLOW_LINE_A_std,SLURRY_FLOW_LINE_A_min,SLURRY_FLOW_LINE_A_max,SLURRY_FLOW_LINE_A_median,SLURRY_FLOW_LINE_A_mad,SLURRY_FLOW_LINE_A_q25,SLURRY_FLOW_LINE_A_q75,AVG_REMOVAL_RATE,STAGE_A,STAGE_B
0,-4226160404,4,0,151.0,245.289474,66.305305,0.0,270.0,268.8,39.205471,...,6.930228,0.000000,26.250000,2.222222,4.524120,2.222222,2.222222,60.44715,1,0
1,-4226160404,4,1,197.0,9.808696,37.368054,0.0,216.0,0.0,17.911531,...,8.079811,2.222222,26.388889,2.361111,6.861741,2.222222,13.402778,60.44715,1,0
2,-4226160404,5,0,84.0,261.967059,41.582836,0.0,270.0,268.8,13.776498,...,0.512545,2.222222,5.277778,2.222222,0.248904,2.222222,2.222222,60.44715,1,0
3,-4226160404,6,0,53.0,250.511111,30.915629,114.0,270.0,268.8,25.551440,...,9.758698,2.222222,26.250000,2.222222,8.592440,2.222222,17.743056,60.44715,1,0
4,-4226160404,6,1,19.0,22.020000,52.495209,0.0,200.4,0.0,35.232000,...,6.841399,2.222222,25.972222,4.166667,5.050000,2.222222,8.194444,60.44715,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1857,4227773662,6,0,73.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,65.07495,1,0
1858,4229773718,4,0,178.0,222.021053,84.233773,0.0,258.0,258.0,55.686550,...,6.959906,2.222222,26.250000,2.222222,4.134264,2.222222,2.222222,73.46535,0,1
1859,4229773718,4,1,210.0,22.290000,56.922871,0.0,216.0,0.0,36.778500,...,7.488812,2.222222,26.250000,3.472222,5.511979,2.222222,7.152778,73.46535,0,1
1860,4229773718,5,0,85.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,1.932004,0.000000,17.916667,0.000000,0.411822,0.000000,0.000000,73.46535,0,1


### 建立模型

In [249]:
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as MSE

In [259]:
X_train = train_data.drop(['WAFER_ID', 'AVG_REMOVAL_RATE'], axis=1).values
y_train = train_data['AVG_REMOVAL_RATE'].values

X_test = test_data.drop(['WAFER_ID', 'AVG_REMOVAL_RATE'], axis=1).values
y_test = test_data['AVG_REMOVAL_RATE'].values

In [269]:
pipe = Pipeline(steps=[
    ('scaler', preprocessing.RobustScaler()),
    ('rf', RandomForestRegressor())])

random_forest_params = {'rf__n_estimators': list(range(50, 300, 15)),
                      'rf__max_depth': list(range(20, 200, 10))}

random_search = RandomizedSearchCV(estimator = pipe,
                                   param_distributions = random_forest_params,
                                   n_iter=50,
                                   n_jobs=-1, cv=7)

In [270]:
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=7,
                   estimator=Pipeline(steps=[('scaler', RobustScaler()),
                                             ('rf', RandomForestRegressor())]),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'rf__max_depth': [20, 30, 40, 50, 60,
                                                          70, 80, 90, 100, 110,
                                                          120, 130, 140, 150,
                                                          160, 170, 180, 190],
                                        'rf__n_estimators': [50, 65, 80, 95,
                                                             110, 125, 140, 155,
                                                             170, 185, 200, 215,
                                                             230, 245, 260, 275,
                                                             290]})

In [272]:
print(random_search.best_score_)
print(random_search.best_params_)

0.8434242209033597
{'rf__n_estimators': 125, 'rf__max_depth': 120}


In [297]:
# 预测
y_pred = random_search.best_estimator_.predict(X_test)
print(MSE(y_test, y_pred))

7.564611041518569


In [301]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)
test_df = test_data[['WAFER_ID', 'CHAMBER', 'DRESSING_WATER_STATUS', 'STAGE_A', 'STAGE_B', 'AVG_REMOVAL_RATE']]
test_df['predict'] = y_pred
test_df

,WAFER_ID,CHAMBER,DRESSING_WATER_STATUS,STAGE_A,STAGE_B,AVG_REMOVAL_RATE,predict
0,-4226160404,4,0,1,0,60.44715,62.207321
1,-4226160404,4,1,1,0,60.44715,62.254050
2,-4226160404,5,0,1,0,60.44715,62.181559
3,-4226160404,6,0,1,0,60.44715,62.890728
4,-4226160404,6,1,1,0,60.44715,62.395446
...,...,...,...,...,...,...,...
1857,4227773662,6,0,1,0,65.07495,63.514954
1858,4229773718,4,0,0,1,73.46535,71.168255
1859,4229773718,4,1,0,1,73.46535,71.669538
1860,4229773718,5,0,0,1,73.46535,71.771010


In [303]:
sum((test_df['AVG_REMOVAL_RATE'] - test_df['predict'])**2) / 1862 

7.564611041518563

In [283]:
# plt.figure(figsize=(8,6))
# plt.plot(range(200), y_pred[0:200], marker='o')
# plt.plot(range(200), y_test[0:200], marker='*')